In [1]:
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10
import torchvision
import numpy as np
import random

import torch
import torch.nn.functional as F
import cl_gym as cl

import sys
import os

seed = 1

np.random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

def make_params() -> dict:
    import os
    from pathlib import Path
    import uuid

    params = {
            # dataset
            'dataset': "MNIST",
            # 'dataset': "FMNIST",

            # benchmark
            'seed': seed,
            'num_tasks': 5,
            'epochs_per_task': 1,
            'per_task_examples': 10000,
            'per_task_examples': np.inf,
            'per_task_memory_examples': 20,
            'batch_size_train': 64,
            'batch_size_memory': 64,
            'batch_size_validation': 256,
            'lambda': 10,

            # algorithm
            'optimizer': 'sgd',
            'learning_rate': 0.001,
            'momentum': 0.9,
            'learning_rate_decay': 1.0,
            'criterion': torch.nn.CrossEntropyLoss(),
            'device': torch.device('cuda:7' if torch.cuda.is_available() else 'cpu'),
             
            # sample selection
            'alpha':0.002
              }

#     trial_id = str(uuid.uuid4())
    trial_id = f"dataset={params['dataset']}/seed={params['seed']}_epoch={params['epochs_per_task']}_lr={params['learning_rate']}_alpha={params['alpha']}_lambda={params['lambda']}"
    params['trial_id'] = trial_id
    params['output_dir'] = os.path.join("./outputs/{}".format(trial_id))
    print(f"output_dir={params['output_dir']}")
    Path(params['output_dir']).mkdir(parents=True, exist_ok=True)

    return params

params = make_params()

output_dir=./outputs/dataset=MNIST/seed=1_epoch=1_lr=0.001_alpha=0.002_lambda=10


In [2]:
from datasets import MNIST
from datasets import FashionMNIST

if params['dataset'] == 'MNIST':
    benchmark = MNIST(num_tasks=params['num_tasks'],
                      per_task_memory_examples=params['per_task_memory_examples'],
                      per_task_examples = min(params['per_task_examples'], 10000),
                      random_class_idx = False)
    label_li = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    n_feature = 28*28

elif params['dataset'] == 'FMNIST':
    benchmark = FashionMNIST(num_tasks=params['num_tasks'],
                             per_task_memory_examples=params['per_task_memory_examples'],
                             per_task_examples = min(params['per_task_examples'], 15000),
                             random_class_idx = False)

    label_li = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 
                  'Ankel boot']
    n_feature = 28*28
    
elif params['dataset'] == 'CIFAR10':
    label_li = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    benchmark = cl.benchmarks.SplitCIFAR10(num_tasks=params['num_tasks'],
                                        per_task_memory_examples=params['per_task_memory_examples'])
    n_feature = 32*32*3

[0 1 2 3 4 5 6 7 8 9]


In [3]:
benchmark.trains[1].inputs.shape

torch.Size([12665, 1, 28, 28])

In [4]:
from algorithms.imbalance import Heuristic2
from metrics import MetricCollector2

backbone = cl.backbones.MLP2Layers(input_dim=784, hidden_dim_1=256, hidden_dim_2=256, output_dim=10)
algorithm = Heuristic2(backbone, benchmark, params, requires_memory=True)
metric_manager_callback = MetricCollector2(num_tasks=params['num_tasks'],
                                                        eval_interval='epoch',
                                                        epochs_per_task=params['epochs_per_task'])

In [5]:
from trainers import ContinualTrainer2

trainer = ContinualTrainer2(algorithm, params, callbacks=[metric_manager_callback])
trainer.run()
print("final avg-acc", metric_manager_callback.meters['accuracy'].compute_final())
print("final avg-forget", metric_manager_callback.meters['forgetting'].compute_final())

---------------------------- Task 1 -----------------------
solver=<function LS_solver at 0x7ff9caa3d280>
[1] Eval metrics for task 1 >> {'accuracy': 99.71478018520183, 'loss': 5.772249579394399e-05, 'std': 0.020902634181430013}
training_task_end
---------------------------- Task 2 -----------------------
solver=<function LS_solver at 0x7ff9caa3d280>
losses=tensor([[0.0135, 0.0136, 6.6504, 7.1469]], device='cuda:7')
n_grads_all.mean(dim=1)=tensor([1.5128e-08, 2.8390e-09, 2.6273e-11, 1.7576e-11], device='cuda:7')
A_np.shape=(4, 10000)
b_np.shape=(4,)
Elapsed time:5.884
Loss difference:[1.30764684e-06 1.30764892e-06 1.30765768e-06 1.30764710e-06]
len(updated_seq_indices)=10000
[2] Eval metrics for task 1 >> {'accuracy': 96.60073721118403, 'loss': 0.0005585125134487242, 'std': 1.358446462285351}
[2] Eval metrics for task 2 >> {'accuracy': 92.10367257655999, 'loss': 0.0010151221224423371, 'std': 3.44088187888556}
training_task_end
---------------------------- Task 3 -----------------------

In [6]:
metric_manager_callback.meters['std'].get_std()

[0.020902634181430013,
 3.44940864927104,
 8.000457215977727,
 11.931833400332684,
 16.356347011705232]

In [7]:
metric_manager_callback.meters['accuracy'].get_data()

array([[99.715,  0.   ,  0.   ,  0.   ,  0.   ],
       [96.601, 92.104,  0.   ,  0.   ,  0.   ],
       [89.394, 75.102, 89.704,  0.   ,  0.   ],
       [80.811, 68.163, 78.883, 96.728,  0.   ],
       [77.572, 53.198, 57.108, 80.882, 92.226]])

In [8]:
metric_manager_callback.meters['accuracy'].compute_overall()

[99.71478018520183,
 94.352204893872,
 84.73311789524782,
 81.14609907784306,
 72.19706602096376]